In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter
import time

In [2]:
f = open('../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

f = open("../../dataset/sense/dict_sense-keys", 'rb')
dict_sense_keys = pickle.load(f)
f.close()

f = open("../../dataset/sense/dict_word-sense", 'rb')
dict_word_sense = pickle.load(f)
f.close()

# Model Description
sense_word = 'serve'
model_name = 'multigpu-2'
sense_word_dir = 'output/' + sense_word
model_dir = sense_word_dir + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(sense_word_dir):
    os.mkdir(sense_word_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

f = open("../../dataset/checkwords/"+ sense_word + "_data", 'rb')
data = pickle.load(f)
f.close()

data_y = []
for i in range(len(data)):
    data_y.append(dict_sense_keys[data[i][0]][2])

sense_count = Counter(data_y)
sense_count = sense_count.most_common()
vocab_sense = [k for k,v in sense_count[:4]]
print(sense_count)

data_x = []
data_label = []
for i in range(len(data)):
    if dict_sense_keys[data[i][0]][2] in vocab_sense:
        data_x.append(data[i][1])
        data_label.append(dict_sense_keys[data[i][0]][2])

print(len(data_label), len(data_y))

# vocab_sense = dict_word_sense[sense_word]

sense2id = dict((s, i) for i,s in enumerate(vocab_sense))
id2sense = dict((i, s) for i,s in enumerate(vocab))

# Parameters
mode = 'train'

num_senses = len(vocab_sense)
batch_size = 32
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = 200
hidden_size = 100
keep_prob = 0.4
l2_lambda = 0.005
init_lr = 0.005
decay_steps = 500
decay_rate = 0.96
clip_norm = 1
clipping = True
moving_avg_deacy = 0.999
num_gpus = 6

index = []
for i in range(len(data_x)):
    index.append(i)

index_train, index_val, label_train, label_val = train_test_split(index, data_label, train_size=0.8, shuffle=True, stratify=data_label, random_state=0)

data_x = np.array(data_x)

x_train = data_x[index_train]
x_val = data_x[index_val]

[('42', 1941), ('41', 839), ('33', 529), ('35', 112), ('34', 41)]
3421 3462


/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
# MODEL

# with tf.device('/cpu:0'):
tower_grads = []
losses = []
predictions = []
x = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [num_gpus, batch_size], name="y")
x_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

with tf.variable_scope("word_embedding"):
    unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)

with tf.variable_scope(tf.get_variable_scope()):
    for gpu_idx in range(num_gpus):
        with tf.name_scope("model_{}".format(gpu_idx)) as scope, tf.device('/gpu:%d' % i):

            if gpu_idx > 0:
                    tf.get_variable_scope().reuse_variables()

            with tf.name_scope("word"):
                Wx = tf.nn.embedding_lookup(final_word_emb_mat, x[gpu_idx])  

            x_len = tf.reduce_sum(tf.cast(x_mask[gpu_idx], 'int32'), 1)

            with tf.variable_scope("lstm1"):
                cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
                d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)

                (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
                h1 = tf.concat([fw_h1, bw_h1], 2)

            with tf.variable_scope("lstm2"):
                cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
                d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)

                (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
                h = tf.concat([fw_h2, bw_h2], 2)

            def attention(input_x, input_mask, W_att):
                h_masked = tf.boolean_mask(input_x, input_mask)
                h_tanh = tf.tanh(h_masked)
                u = tf.matmul(h_tanh, W_att)
                a = tf.nn.softmax(u)
                c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
                return c

            with tf.variable_scope("attention"):
                W_att = tf.get_variable("W_att", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
                c = tf.expand_dims(attention(h[0], x_mask[gpu_idx][0], W_att), 0)
                for i in range(1, batch_size):
                    c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[gpu_idx][i], W_att), 0)], 0)

            with tf.variable_scope("softmax_layer"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_senses], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
                b = tf.get_variable("b", shape=[num_senses], initializer=tf.zeros_initializer())
                drop_c = tf.nn.dropout(c, input_keep_prob)
                logits = tf.matmul(drop_c, W) + b
                predictions.append(tf.argmax(logits, 1))

            loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y[gpu_idx]))

            l2_loss = l2_lambda * tf.losses.get_regularization_loss()

            total_loss = loss + l2_loss
            tf.summary.scalar("loss_{}".format(gpu_idx), total_loss)

            summaries = tf.get_collection(tf.GraphKeys.SUMMARIES, scope)

            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_vars = optimizer.compute_gradients(total_loss)

            clipped_grads = grads_vars
            if(clipping == True):
                clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

            tower_grads.append(clipped_grads)
            losses.append(total_loss)

tower_grads = average_gradients(tower_grads)
losses = tf.add_n(losses)/len(losses)
apply_grad_op = optimizer.apply_gradients(tower_grads, global_step=global_step)
summaries.append(tf.summary.scalar('learning_rate', learning_rate))

for var in tf.trainable_variables():
    summaries.append(tf.summary.histogram(var.op.name, var))

variable_averages = tf.train.ExponentialMovingAverage(moving_avg_deacy, global_step)
variables_averages_op = variable_averages.apply(tf.trainable_variables())

train_op = tf.group(apply_grad_op, variables_averages_op)
saver = tf.train.Saver(tf.global_variables())
summary = tf.summary.merge(summaries)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# print (device_lib.list_local_devices())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [6]:
def data_prepare(x, y):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    yy = np.zeros([num_examples], dtype=int)

    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
        yy[j] = sense2id[y[j]]
    return xx, xx_mask, yy

def eval_score(yy, pred):
    num_batches = int(len(yy)/(batch_size*num_gpus))
    f1 = f1_score(yy[:batch_size*num_batches*num_gpus], pred, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches*num_gpus], pred)
    return f1*100, accu*100

def model(xx, yy, mask, train_cond=True):
    num_batches = int(len(xx)/(batch_size*num_gpus))
    _losses = 0
    preds = []
    
    for j in range(num_batches): 
        
        s = j * batch_size * num_gpus
        e = (j+1) * batch_size * num_gpus
        xx_re = xx[s:e].reshape([num_gpus, batch_size, -1])
        yy_re = yy[s:e].reshape([num_gpus, batch_size])
        mask_re = mask[s:e].reshape([num_gpus, batch_size, -1])
        
        feed_dict = {x:xx_re, y:yy_re, x_mask:mask_re, is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, losses, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)  
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred = sess.run([total_loss, predictions], feed_dict)
            for i in range(num_gpus):
                preds.append(pred[i])
            
            
        _losses +=_loss

    if(train_cond==False):
        y_pred = []
        for pred in preds:
            for bt in pred:
                y_pred.append(bt)
        return _losses/num_batches, y_pred
    
    return _losses/num_batches, step

In [7]:
x_id_train, mask_train, y_train = data_prepare(x_train, label_train)
x_id_val, mask_val, y_val = data_prepare(x_val, label_val)

In [8]:
num_epochs = 60
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    
    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, mask_train)
    time_taken = time.time() - start_time
    print("Epoch:", i+1,"Step:", step, "loss:{0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))
    
    if((i+1)%log_period==0):
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        start_time = time.time()
        train_loss, train_pred = model(x_id_train, y_train, mask_train, train_cond=False)
        f1_, accu_ = eval_score(y_train, train_pred)
        time_taken = time.time() - start_time
        print("Train: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), "Loss:{0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))
        
        start_time = time.time()
        val_loss, val_pred = model(x_id_val, y_val, mask_val, train_cond=False)
        f1_, accu_ = eval_score(y_val, val_pred)
        time_taken = time.time() - start_time
        print("Val: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.4f}".format(time_taken))
        

Epoch: 1 Step: 14 loss:9.7987 , Time: 112.6370
Epoch: 2 Step: 28 loss:1.2703 , Time: 106.6000
Epoch: 3 Step: 42 loss:0.9235 , Time: 108.2051
Epoch: 4 Step: 56 loss:0.8701 , Time: 108.3720
Epoch: 5 Step: 70 loss:0.7698 , Time: 108.6257
Model Saved
Train: F1 Score:60.4399 Accuracy:70.1265 Loss:0.8166 , Time: 74.4978
Val: F1 Score:53.2502 Accuracy:68.2292 Loss:0.6735 , Time: 15.8553
Epoch: 6 Step: 84 loss:0.7341 , Time: 109.8029
Epoch: 7 Step: 98 loss:0.7167 , Time: 106.7240
Epoch: 8 Step: 112 loss:0.6862 , Time: 107.4377
Epoch: 9 Step: 126 loss:0.6861 , Time: 108.5872
Epoch: 10 Step: 140 loss:0.6502 , Time: 108.0874
Model Saved
Train: F1 Score:72.3512 Accuracy:75.8557 Loss:0.6408 , Time: 72.8027
Val: F1 Score:64.4819 Accuracy:72.2222 Loss:0.7323 , Time: 15.7853
Epoch: 11 Step: 154 loss:0.6147 , Time: 108.1566
Epoch: 12 Step: 168 loss:0.6061 , Time: 108.2515
Epoch: 13 Step: 182 loss:0.5773 , Time: 106.0049
Epoch: 14 Step: 196 loss:0.5841 , Time: 107.6694
Epoch: 15 Step: 210 loss:0.5719 , 

In [ ]:
saver.restore(sess, save_dir)